# Exploring Hacker News Posts

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- id: The unique identifier from Hacker News for the post
- title: The title of the post
- url: The URL that the posts links to, if it the post has a URL
- num_points: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- num_comments: The number of comments that were made on the post
- author: The username of the person who submitted the post
- created_at: The date and time at which the post was submitted

## Introduction

We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question.

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting. 

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

Let's start by importing the libraries we need and reading the data set into a list of lists.

In [1]:
from csv import reader 
open_file = open('hacker_news.csv', encoding="utf8")
read_file = reader(open_file)
hn = list(read_file)
print(hn[:5])

[['', 'id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['0', '12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['1', '10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['2', '11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['3', '11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


Now we will seperate the header from the data so that we can manipulate the easily.

In [16]:
headers = hn[0]
hn = hn[1:]
print(hn[:5])

[['0', '12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['1', '10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['2', '11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['3', '11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['4', '10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting Ask HN and Show HN Posts

Now that we've removed the headers from hn, we're ready to filter our data. Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [22]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[2]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print('The number of Ask posts is: ',len(ask_posts))
print('The number of Show posts is: ',len(show_posts))
print('The number of Other posts is: ',len(other_posts))

The number of Ask posts is:  1744
The number of Show posts is:  1162
The number of Other posts is:  17194


As we can see from the results above, the majority of the posts are neither Ask posts nor Show posts. However, the number of Ask posts does exceed the number of Show posts.

## Calculating the Average Number of Comments for Ask HN and Show HN Posts


Next, let's determine if ask posts or show posts receive more comments on average.



In [26]:
# Computing the average number of comments in the Ask hn:

total_ask_comments = 0
for row in ask_posts:
    comments = int(row[5])
    total_ask_comments += comments
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

# Computing the average number of comments in the Show hn:

total_show_comments = 0
for row in show_posts:
    comments = int(row[5])
    total_show_comments += comments
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

14.038417431192661
10.31669535283993


On average, the Ask section receives 14 comments per post while the Show section receives 10 comments per post.

Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

In [66]:
import datetime as dt

# Creating a list with the post creation time and the number of commnents for
# the ask _posts

result_list = []
for row in ask_posts:
    creation_date = row[7]
    comments = int(row[5])
    result_list.append([creation_date,comments])
    
# Getting the number of posts per hour along with the number of comments
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    comments = int(row[1])
    hour = dt.datetime.strptime(row[0],'%m/%d/%Y %H:%M').strftime('%H')
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = comments
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += comments
print('Average posts per hour:')
counts_by_hour

Average posts per hour:


{'00': 55,
 '01': 60,
 '02': 58,
 '03': 54,
 '04': 47,
 '05': 46,
 '06': 44,
 '07': 34,
 '08': 48,
 '09': 45,
 '10': 59,
 '11': 58,
 '12': 73,
 '13': 85,
 '14': 107,
 '15': 116,
 '16': 108,
 '17': 100,
 '18': 109,
 '19': 110,
 '20': 80,
 '21': 109,
 '22': 71,
 '23': 68}

In [74]:
# Calculate the average number of comments ask posts receive by hour created

avg_by_hour = []
for hour in comments_by_hour:
    avg_by_hour.append([hour,comments_by_hour[hour] / counts_by_hour[hour]])
print('Average number of comments per posts for each hour:')
avg_by_hour

Average number of comments per posts for each hour:


[['11', 11.051724137931034],
 ['17', 11.46],
 ['02', 23.810344827586206],
 ['21', 16.009174311926607],
 ['01', 11.383333333333333],
 ['08', 10.25],
 ['05', 10.08695652173913],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['09', 5.5777777777777775],
 ['14', 13.233644859813085],
 ['22', 6.746478873239437],
 ['04', 7.170212765957447],
 ['06', 9.022727272727273],
 ['15', 38.5948275862069],
 ['03', 7.796296296296297],
 ['12', 9.41095890410959],
 ['07', 7.852941176470588],
 ['19', 10.8],
 ['18', 13.20183486238532],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['20', 21.525],
 ['00', 8.127272727272727]]

## Sorting and Printing Values from a List of Lists


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.



In [122]:
# swap the first and second columns in order to sort the list

swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
swap_avg_by_hour

# Sort the results

sorted_swap = sorted(swap_avg_by_hour, reverse = True)
    
sorted_swap

# Print the top 5 hours for Ask post comments
for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1],'%H') + dt.timedelta(hours = 7)
    hour = hour.time().strftime('%H:%M')
    avg = row[0]
    print('{}: {:.2f} average comments per post'.format(hour,avg))
    



22:00: 38.59 average comments per post
09:00: 23.81 average comments per post
03:00: 21.52 average comments per post
23:00: 16.80 average comments per post
04:00: 16.01 average comments per post


From the results above, we can conclude that the hours 22:00, 09:00, 03:00, 23:00, and 04:00, in Beirut local time, have a higher chance of receiving comments.

## Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 22:00 and 24:00 Beirut local time.